# butcher2

The Markov Butcher Shop sells premium grade A+ beef. Customers arrive at the butcher shop and form a single queue. There is one butcher who serves customers from the queue on a first-come-first-served basis.

Based on historical data, the interarrival time between customers is uniformly distributed between 1.5 and 3.5 minutes. The time for a butcher to fill a single order is uniformly distributed between 2.5 and 6.5 minutes. The interarrival times and service times are assumed to be independent.

a. Assume there is 1 butcher. Simulate the shop for 2 hours.

In [1]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform


##### Parameters #####
class P:
    # Customers arrive at the entrance with uniformly distributed
    # interarrival times between 1.5 and 3.5 minutes
    interarrivalTimeMin = 1.5
    interarrivalTimeMax = 3.5
    
    # Service times are uniformly distributed between 2.5 and 6.5 minutes
    serviceTimeMin = 2.5
    serviceTimeMax = 6.5
    
    # Number of servers
    nServers = 1
    
    # Shop is open for 2 continuous hours
    simulationTimeMax =  2 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = uniform(low = P.interarrivalTimeMin, high = P.interarrivalTimeMax)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity=P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until=P.simulationTimeMax)
    

##### Experiment #####
model()

Time 2.892938371195723: Customer 0 arrives and joins queue
Time 2.892938371195723: Customer 0 begins service
Time 4.965217041096482: Customer 1 arrives and joins queue
Time 6.300344185452536: Customer 0 ends service and leaves
Time 6.300344185452536: Customer 1 begins service
Time 7.567846579262264: Customer 2 arrives and joins queue
Time 9.914059499511186: Customer 3 arrives and joins queue
Time 11.678220064594788: Customer 1 ends service and leaves
Time 11.678220064594788: Customer 2 begins service
Time 13.375587896280418: Customer 4 arrives and joins queue
Time 15.83745169924914: Customer 5 arrives and joins queue
Time 16.917539018934242: Customer 2 ends service and leaves
Time 16.917539018934242: Customer 3 begins service
Time 18.12168673563744: Customer 6 arrives and joins queue
Time 20.79025108353772: Customer 3 ends service and leaves
Time 20.79025108353772: Customer 4 begins service
Time 21.07978615040552: Customer 7 arrives and joins queue
Time 22.699141943624657: Customer 8 a

b. How many customers arrive at the shop over the 2 hour period?

47 customers arrive (Customers 0 to 46).

c. Looking ahead: How would you change your model so that there are 2 butchers? 3? $n$? Read the SimPy documentation to find out.

You should find [this section](http://simpy.sourceforge.net/old/SManual/SManual.html#resources) of the documentation useful.